In [1]:
from azureml.core import Workspace, Experiment

####ws = Workspace.get(name="quick-starts-ws-222154")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

C:\Users\Viet\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Workspace name: quick-starts-ws-222154
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-222154


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

##cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Specify the configuration for the new cluster
    ##compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', max_nodes=4)
    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(workspace=ws,
                                       name=cluster_name, 
                                       provisioning_configuration=compute_config)

# Wait for the cluster to complete, show the output log
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute cluster...
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-01-10T12:10:57.046000+00:00', 'errors': None, 'creationTime': '2023-01-10T12:10:51.817568+00:00', 'modifiedTime': '2023-01-10T12:10:58.499617+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

######## My Code ######
import shutil
#######################

# Specify parameter sampler
##ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
            {
                '--C': choice(0.01, 0.1, 1.0, 10.0, 100.0),
                '--max_iter': choice(20, 50, 100, 120, 150)
            }
        )


# Specify a Policy
##policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

######## My Code ######
train_script = "./training"
shutil.copy('train.py', train_script)
#######################

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
##src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory=train_script, 
                      script='train.py', 
                      compute_target=compute_target, 
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
##hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=ps, 
                                     policy=policy, 
                                     primary_metric_name='validation_acc',            # Accuracy
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=4,                                # 20, 100
                                     max_concurrent_runs=4)                           # 20

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

# Start the HyperDrive run
hyperdrive_run = exp.submit(config = hyperdrive_config, show_output = True)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

# Wait for the cluster to complete, show the output log
hyperdrive_run.wait_for_completion(show_output=True)

# evaluate the the run is indeed complete
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fedcaf80-926d-41f8-99df-4aed00423ab1
Web View: https://ml.azure.com/runs/HD_fedcaf80-926d-41f8-99df-4aed00423ab1?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-222154/workspaces/quick-starts-ws-222154&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-10T12:22:53.002003][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-10T12:22:53.9690216Z][SCHEDULER][INFO]Scheduling job, id='HD_fedcaf80-926d-41f8-99df-4aed00423ab1_1' 
[2023-01-10T12:22:54.0787224Z][SCHEDULER][INFO]Scheduling job, id='HD_fedcaf80-926d-41f8-99df-4aed00423ab1_2' 
[2023-01-10T12:22:54.190745][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-01-10T12:22:54.2803570Z][SCHEDULER][INFO]Scheduling job, id='HD_fedcaf80-926d-41f8-99df-4aed00423ab1_3' 
[2023-01-10T12:22:54.6138195Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_fedcaf80-926d-41f8-9

In [17]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False)) # top=1

# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

########################################
# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
#print("Best run metrics :",best_run.get_metrics())
#best_run_metrics = best_run.get_metrics()
#print('Accuracy:', best_run_metrics['Accuracy'])

# check metrics details
#best_run_metrics

#### --> object has no attribute 'get_metrics'
########################################

########################################
# get_details()
# Returns a dictionary with the details for the run
#print("Best run details :",best_run.get_details())
#print(best_run.get_details()['runDefinition']['arguments'])

#### --> object has no attribute 'get_details'
########################################

########################################
# get_file_names()
# Returns a list of the files that are stored in association with the run.
#print("Best run file names :",best_run.get_file_names()) # get name of files of best_run

#### --> object has no attribute 'get_file_names'
########################################

########################################
# download the best run and register the model
#best_run.download_file(name=train_script+'/model.joblib', 
#                       output_file_path=train_script)

#### --> object has no attribute 'download_file'
########################################

########################################
# save the model, i.e., output file of best_run
#model = best_run.register_model(model_name='hyperdrive_run', 
#                                model_path=train_script+'/model.joblib')

#### --> object has no attribute 'register_model'
########################################

# save file
os.makedirs('./outputs',exist_ok = True)
joblib.dump(value=best_run, filename = 'outputs/bestHDModel.txt')
joblib.dump(value=best_run, filename='outputs/bankmarketing_model.pkl')

[{'run_id': 'HD_fedcaf80-926d-41f8-99df-4aed00423ab1_3', 'hyperparameters': '{"--C": 0.01, "--max_iter": 150}', 'best_primary_metric': None, 'status': 'Completed'}, {'run_id': 'HD_fedcaf80-926d-41f8-99df-4aed00423ab1_0', 'hyperparameters': '{"--C": 0.01, "--max_iter": 100}', 'best_primary_metric': None, 'status': 'Completed'}, {'run_id': 'HD_fedcaf80-926d-41f8-99df-4aed00423ab1_1', 'hyperparameters': '{"--C": 0.1, "--max_iter": 120}', 'best_primary_metric': None, 'status': 'Completed'}, {'run_id': 'HD_fedcaf80-926d-41f8-99df-4aed00423ab1_2', 'hyperparameters': '{"--C": 10.0, "--max_iter": 20}', 'best_primary_metric': None, 'status': 'Completed'}]


['outputs/bankmarketing_model.pkl']

In [20]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=url)

# create a dataframe with ds data
ds_df = dataset.to_pandas_dataframe()
ds_df.head()
ds_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32950 non-null  int64  
 1   job             32950 non-null  object 
 2   marital         32950 non-null  object 
 3   education       32950 non-null  object 
 4   default         32950 non-null  object 
 5   housing         32950 non-null  object 
 6   loan            32950 non-null  object 
 7   contact         32950 non-null  object 
 8   month           32950 non-null  object 
 9   day_of_week     32950 non-null  object 
 10  duration        32950 non-null  int64  
 11  campaign        32950 non-null  int64  
 12  pdays           32950 non-null  int64  
 13  previous        32950 non-null  int64  
 14  poutcome        32950 non-null  object 
 15  emp.var.rate    32950 non-null  float64
 16  cons.price.idx  32950 non-null  float64
 17  cons.conf.idx   32950 non-null 

In [23]:
from train import clean_data

##### My Code ################
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core.dataset import Dataset
##############################

# Use the clean_data function to clean your data.
##x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(dataset)

dataset.take(3).to_pandas_dataframe()

# split data into Train and Test Sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# combine the training features and the label
train_df = pd.concat([x_train, y_train.to_frame(name="label")], axis=1)
validation_df = pd.concat([x_test, y_test.to_frame(name="label")], axis=1)

train_df.head()
train_df.info()

if not os.path.isdir('automl_data'):
    os.mkdir('automl_data')

# Save the train and validation data to a csv to be uploaded to the datastore
train_df.to_csv("automl_data/train_data.csv", index=False)
validation_df.to_csv("automl_data/validation_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./automl_data', target_path='bankmarketing', overwrite=True, show_progress=True)

# Upload the training data as a tabular dataset for access during training on remote compute
############train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
# validation_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))

label = "label"
############train_data.take(3).to_pandas_dataframe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26360 entries, 26529 to 2732
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            26360 non-null  int64  
 1   marital                        26360 non-null  int64  
 2   default                        26360 non-null  int64  
 3   housing                        26360 non-null  int64  
 4   loan                           26360 non-null  int64  
 5   month                          26360 non-null  int64  
 6   day_of_week                    26360 non-null  int64  
 7   duration                       26360 non-null  int64  
 8   campaign                       26360 non-null  int64  
 9   pdays                          26360 non-null  int64  
 10  previous                       26360 non-null  int64  
 11  poutcome                       26360 non-null  int64  
 12  emp.var.rate                   26360 non-nu

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Viet\Anaconda3\lib\site-packages\azureml\data\azure_storage_datastore.py", line 353, in handler
    result = future.result()
  File "C:\Users\Viet\Anaconda3\lib\concurrent\futures\_base.py", line 439, in result
    return self.__get_result()
  File "C:\Users\Viet\Anaconda3\lib\concurrent\futures\_base.py", line 391, in __get_result
    raise self._exception
  File "C:\Users\Viet\Anaconda3\lib\concurrent\futures\thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "C:\Users\Viet\Anaconda3\lib\site-packages\azureml\data\azure_storage_datastore.py", line 934, in <lambda>
    lambda target, source: lambda: [(self.blob_service.get_blob_client(
  File "C:\Users\Viet\Anaconda3\lib\site-packages\azureml\data\azure_storage_datastore.py", line 934, in <listcomp>
    lambda target, source: lambda: [(self.blob_service.get_blob_client(
  File "C:\Users\Viet\Anaconda3\lib\site-packages\azure\core\

In [30]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
'''
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)
'''

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_df,
    label_column_name='y',
    n_cross_validations=5)


ModuleNotFoundError: No module named 'azureml.train.automl'

In [2]:
######## My Code ######
from azureml.widgets import RunDetails
#######################

# Submit your automl run

### YOUR CODE HERE ###

automl_run = exp.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

# Wait to complete, show the output log
####automl_run.wait_for_completion()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

# get name of files of best_run
best_run.get_file_names()
print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
best_run.get_metrics()

fitted_model
best_run

# Cluster clean up
compute_target.delete()